In [153]:
import pandas as pd
null="\u03bb"

In [283]:
# reading DFA from file
inputFile="C:/Users/Admin/Desktop/FA CSV input files/DFA8.csv"
DFA = pd.read_csv(inputFile, index_col="states", dtype="str") 
DFA.index = DFA.index.map(str)
print(DFA)


# convert data to appropriate type (using frozen set for states as they are hashable and can be used as index)
DFA.isInitial = DFA["isInitial"].map({'FALSE':False, 'TRUE':True}) # bool
#NFA.index = [frozenset(x) for x in NFA.index] # bool
DFA.isFinal = DFA["isFinal"].map({'FALSE':False, 'TRUE':True}) # bool

# find final states, intial state  and symbols 
q0= DFA.loc[DFA["isInitial"]==True].index.values[0]
print("inital Stated of NFA=",q0)
finalStates=set(DFA.loc[DFA["isFinal"]==True].index.values.tolist())
print("final states: ", finalStates)
symbols= [x for x in list(DFA.columns) if x.find("input")!=-1]
print("symbols: ", symbols)
states= DFA.index.values.tolist()
print("States", states)



       input_a input_b isInitial isFinal
states                                  
1            2       2      TRUE    TRUE
2            3       4     FALSE    TRUE
3          NaN     NaN     FALSE    TRUE
4            2     NaN     FALSE   FALSE
inital Stated of NFA= 1
final states:  {'3', '2', '1'}
symbols:  ['input_a', 'input_b']
States ['1', '2', '3', '4']


In [280]:
# create table to store transitions between each pair of state
table= pd.DataFrame(index=states+["NI","NF"], columns=states+["NI","NF"])

# add tran(NI,null)=q0 and tran(finalstates, null)=NF
table.loc["NI",q0]= null
table.loc[finalStates,"NF"]= null


# add the intitial transitions from given DFA to table
for x in states: # sourse
    for s in symbols: # symbol
        if DFA.loc[x, s]==DFA.loc[x, s]:  #destination if not nan
            if table.loc[x,DFA.loc[x, s]]==table.loc[x,DFA.loc[x, s]]: #initially table location is not nan
                # add this transition to prevous transition (union) with brackets
                table.loc[x,DFA.loc[x, s]]= "(" + s.replace("input_","") + " + "+table.loc[x,DFA.loc[x, s]]+")"
            else:
                table.loc[x,DFA.loc[x, s] ]= s.replace("input_","")           
print(table)


       
            

      1        2    3    4   NI   NF
1   NaN  (b + a)  NaN  NaN  NaN    λ
2   NaN      NaN    a    b  NaN    λ
3   NaN      NaN  NaN  NaN  NaN    λ
4   NaN        a  NaN  NaN  NaN  NaN
NI    λ      NaN  NaN  NaN  NaN  NaN
NF  NaN      NaN  NaN  NaN  NaN  NaN


In [281]:

# eliminate states one by one
# states will be elimiated in order of their occurance in DFA index
for x in states:
    print("Eliminating state ",x)
    
    loop= "("+table.loc[x,x] +")*" if table.loc[x,x]==table.loc[x,x] else ""     
    outward=[y for y in table.loc[(table.loc[x].isna()==False)].index.values if y!=x]
    inward=[y for y in table.loc[table.loc[states+["NI","NF"],x].isna()==False].index.values if y!=x]
    print("loop on state", x, (loop if loop!="" else "-NA-"))
    
    print("outward transitions",outward)
    print("inward transitions",inward)
    
    for i in inward:
        for o in outward:
            
            
            
            REin= (table.loc[i,x] if table.loc[i,x]==table.loc[i,x] else "")
            REout=(table.loc[x,o] if table.loc[x,o]==table.loc[x,o] else "")
            REold=(table.loc[i,o] if table.loc[i,o]==table.loc[i,o] else "")
            
            print("transition from ",i,"to", o, "through", x, "is", REin + loop + REout  +(" + "+ REold if REold!="" else "")
                                                                                           ,"as",
                  "\n  RE from ", i , "to", x, "is ",(REin if REin!="" else "-NA-"),
                  "\n  RE of loop is ", (loop if loop!="" else "-NA-") , 
                  "\n  RE from ", x , "to", o, "is ",(REout if REout!="" else "-NA-"),
                  "\n  old RE from ", i , "to", o, "is ",(REold if REold!="" else "-NA-"))
            if (REin==null and REout!=""):
                REin=""
            if (REout==null and REin!=""):
                REout=""    
            
            
            table.loc[i,o]= REin + loop + REout  +(" + "+ REold if REold!="" else "")
            #table.loc[i,o]=" ( "+(table.loc[i,x]+loop+table.loc[x,o])+ (" + "+table.loc[i,o] if table.loc[i,o]==table.loc[i,o] else "")+" ) "
    table = table.drop(x, 1)
    table = table.drop(x, 0)
    #print(table) 

Eliminating state  1
loop on state 1 -NA-
outward transitions ['2', 'NF']
inward transitions ['NI']
transition from  NI to 2 through 1 is λ(b + a) as 
  RE from  NI to 1 is  λ 
  RE of loop is  -NA- 
  RE from  1 to 2 is  (b + a) 
  old RE from  NI to 2 is  -NA-
transition from  NI to NF through 1 is λλ as 
  RE from  NI to 1 is  λ 
  RE of loop is  -NA- 
  RE from  1 to NF is  λ 
  old RE from  NI to NF is  -NA-
Eliminating state  2
loop on state 2 -NA-
outward transitions ['3', '4', 'NF']
inward transitions ['4', 'NI']
transition from  4 to 3 through 2 is aa as 
  RE from  4 to 2 is  a 
  RE of loop is  -NA- 
  RE from  2 to 3 is  a 
  old RE from  4 to 3 is  -NA-
transition from  4 to 4 through 2 is ab as 
  RE from  4 to 2 is  a 
  RE of loop is  -NA- 
  RE from  2 to 4 is  b 
  old RE from  4 to 4 is  -NA-
transition from  4 to NF through 2 is aλ as 
  RE from  4 to 2 is  a 
  RE of loop is  -NA- 
  RE from  2 to NF is  λ 
  old RE from  4 to NF is  -NA-
transition from  NI to 3 t

c:\python35\lib\site-packages\pandas\core\indexing.py:1020: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [282]:
print("Regular expression for the given FA is \n",table.loc["NI","NF"])
print("Note the the this is only one possible RE, results will be different if states are elimimated in different")

Regular expression for the given FA is 
 (b + a)b(ab)*aa + a + (b + a)a + (b + a) + λ
Note the the this is only one possible RE, results will be different if states are elimimated in different
